### Version information

In [1]:
import sys
import matplotlib.pyplot as plt
sys.path.append(r"C:\Users\LAP21\Documents\pyleecan-public")
from datetime import date
print("Running date:", date.today().strftime("%B %d, %Y"))
import pyleecan
print(f'Pyleecan version : {pyleecan.__version__}')
import SciDataTool
print(f'SciDataTool version : {SciDataTool.__version__}')

Running date: October 24, 2022
Pyleecan version : 1.4.0
SciDataTool version : 2.5.0


## Efficiency map of the machine

The efficiency map of the machine can be constructed by going through several steps.

The next step is to define Simulation containing a *var_simu* attribute with a *VarLoadCurrent*. The *VarLoadCurrent* object needs an *OP_matrix* to define several operating points to carry out multiple simulations. The *OP_matrix* is a matrix with 3 columns representing the following quantities :
1. Rotation speed
2. $I_d$
3. $I_q$

In [ ]:
from pyleecan.Classes.VarLoadCurrent import VarLoadCurrent

simu = Simu1(name="test_ElecLUTdq_efficiency_map", machine=machine)

Nspeed = 50

OP_matrix = np.zeros((Nspeed, 3))
OP_matrix[:, 0] = np.linspace(500, 8000, Nspeed)


simu.input = InputCurrent(
    OP=OPdq(),
    Nt_tot=4 * 8,
    Na_tot=200 * 8,
    is_periodicity_a=True,
    is_periodicity_t=True,
)

Pyleecan allows to define a list of DataKeepers. These objects tell the simulation which output to store, and to throw away the others in order to save memory. A we will run many simulations, it is a goood practice to use them.

In [ ]:
from pyleecan.Classes.DataKeeper import DataKeeper

datakeeper_list=[
    DataKeeper(
        name = "Torque",
        unit = "N.m", 
        symbol = "T",
        keeper = lambda output: output.elec.Tem_av,
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "Efficiency",
        unit = "", 
        symbol = "eff",
        keeper = lambda output: output.elec.OP.efficiency,
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "current density",
        unit = "A/m^2", 
        symbol = "J",
        keeper = lambda output : output.elec.get_Jrms(),
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "Ud",
        unit = "V", 
        symbol = "Ud",
        keeper = lambda output: output.elec.OP.Ud_ref,
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "Uq",
        unit = "V", 
        symbol = "Uq",
        keeper = lambda output: output.elec.OP.Uq_ref,
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "U0",
        unit = "V", 
        symbol = "U0",
        keeper = lambda output: output.elec.OP.get_U0_UPhi0()["U0"] ,
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "I0",
        unit = "A", 
        symbol = "I0",
        keeper = lambda output: output.elec.OP.get_I0_Phi0()["I0"] ,
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "Phid",
        unit = "Wb", 
        symbol = "Phid",
        keeper = lambda output: output.elec.eec.Phid,
        error_keeper = lambda simu: np.nan
    ),
    DataKeeper(
        name = "Phiq",
        unit = "Wb", 
        symbol = "Phiq",
        keeper = lambda output: output.elec.eec.Phiq,
        error_keeper = lambda simu: np.nan
    ),
]

This DataKeeper list is now used in a VarLoadCurrent object.

In [ ]:
OP_matrix = np.zeros((Nspeed, 3))
OP_matrix[:, 0] = np.linspace(500, 6000, Nspeed)
simu.var_simu = VarLoadCurrent(
    datakeeper_list = datakeeper_list
)
simu.var_simu.set_OP_array(OP_matrix, "N0", "Id", "Iq")

The electrical module of the simulation needs to be an ElecLUTdq object to take into account the LUT previously defined.

In [ ]:
from pyleecan.Classes.ElecLUTdq import ElecLUTdq
from pyleecan.Classes.PostLUT import PostLUT
from pyleecan.Classes.Loss import Loss

# Definition of a loss model

simu.elec = ElecLUTdq(
        Urms_max=230,  # Maximum rms phase voltage
        Jrms_max=27e6,  # Maximum rms current density in slot
        n_interp=100,  # Number of id values used for interpolation
        n_Id=5,  # Number of Id for LUT calculation
        n_Iq=5,  # Number of Id for LUT calculation
        Id_max=0,  # Maximum Id for LUT calculation
        Iq_min=0,  # Minimum Iq for LUT calculation
        LUT_enforced=None,  # To use previously computed LUT
        is_grid_dq=True,  # True to build a n_Id*n_Iq grid, otherwise calculate n_Id+n_Iq simulations and extrapolate to the dq plane
        Tsta=120,  # Average stator temperature for Electrical calculation [degC]
        type_skin_effect=1,
        LUT_simu=Simu1(
            input=InputCurrent(
                OP=OPdq(),
                Nt_tot=4 *10 *8,
                Na_tot=200 * 8,
                is_periodicity_a=True,
                is_periodicity_t=True,
            ),
            var_simu=VarLoadCurrent(
                postproc_list=[PostLUT(is_save_LUT=True, file_name = "LUT_eff_Toyota_Prius.h5")],
                is_keep_all_output=True,
            ),
            mag=MagFEMM(
                is_periodicity_a=True,
                is_periodicity_t=True,
                nb_worker=4,
                is_get_meshsolution=True,
            ),
            loss = Loss(
                is_get_meshsolution=False,
                Tsta=100,
                model_dict={"stator core": LossModelSteinmetz(group = "stator core"),
                            "rotor core": LossModelSteinmetz(group = "rotor core"),
                            "joule": LossModelWinding(group = "stator winding"),
                            "proximity": LossModelProximity(group = "stator winding"),
                            "magnets": LossModelMagnet(group = "rotor magnets")}
            )
        ),
    )

### Defining a Look up table (LUT)

A look up table is first needed to allow faster computation  during all the simulations. This LUT stores several quantities with respect to the currents $I_d$ and $I_q$.

The LUT can be stored so it is not calculated each time the scipt is run. If this is the first time and the LUT does not exists, it is set to None and will be computed before running other simulations.

In [ ]:
from os.path import exists, split
from pyleecan.Functions.Load.load_json import LoadMissingFileError
from pyleecan.definitions import RESULT_DIR

path_to_LUT = r"C:\Users\LAP10\Documents\Loss\LUT_nb.h5"
LUT_file_name = f"LUT_eff_Toyota_Prius.h5"
path_to_LUT = join(RESULT_DIR, LUT_file_name)

try:
    LUT_enforced = load(path_to_LUT)
    simu.elec.LUT_enforced = LUT_enforced
except (FileNotFoundError, LoadMissingFileError):
    print("The LUT could not be loaded, so it will be computed.")
    LUT_enforced = None


Now comes the core of the algorithms. For several values of load rates (defining the required torque), a simlation will be run for every values of speed in the *OP_matrix*. The simulations will be very fast as the magnetic and loss quantities are interpolated from the LUT previously computed.

In [ ]:


# Nload simulations will be carried out with load values between 0.1 and 1
Nload = 7
load_vect = np.linspace(0, 1, Nload)

# Several matrixes that will contain relevant data for post_processing are created
OP_matrix_MTPA = np.zeros((Nspeed, Nload, 6))
U_MTPA = np.zeros((Nspeed, Nload, 3))
I_MTPA = np.zeros((Nspeed, Nload, 3))
Phidq_MTPA = np.zeros((Nspeed, Nload, 2))

# An output list is created to contain the Nload outputs that will be computed
out_load = list()


for ii, load_rate in enumerate(load_vect):
    # Re using the LUT computed during the first step
    if ii > 0 and LUT_enforced is None:
            simu.elec.LUT_enforced = load(path_to_LUT)

    simu.elec.load_rate = load_rate

    out = simu.run()

    # Store values in MTPA, the five columns of the OP_matrix_MTPA will be filled
    # First column is rotational speed
    OP_matrix_MTPA[:, ii, 0] = out["N0"].result
    # Second columns is Id
    OP_matrix_MTPA[:, ii, 1] = out["Id"].result
    # Third column is Iq
    OP_matrix_MTPA[:, ii, 2] = out["Iq"].result
    # Fourth column is the average torque
    OP_matrix_MTPA[:, ii, 3] = out["T"].result
    # Fifth column is the efficiency
    OP_matrix_MTPA[:, ii, 4] = out["eff"].result
    # Sixth column is the current density
    OP_matrix_MTPA[:, ii, 5] = out["J"].result
    
    
    # Store the voltages and currents in two dedicated matrixes
    # First column is Ud
    U_MTPA[:, ii, 0] = out["Ud"].result
    # Second column is Uq
    U_MTPA[:, ii, 0] = out["Ud"].result
    # Third column is U0
    U_MTPA[:, ii, 2] = out["U0"].result
    # First column is Id
    I_MTPA[:, ii, 0] = out["Id"].result
    # Second column is Iq
    I_MTPA[:, ii, 1] = out["Iq"].result
    # Third column is I0
    I_MTPA[:, ii, 2] = out["I0"].result
    
    # The fluxes are also stored in their dedicated matrix
    # First column is phid
    Phidq_MTPA[:, ii, 0] = out["Phid"].result
    # second column is phiq
    Phidq_MTPA[:, ii, 1] = out["Phiq"].result
    
    # The output of the current simulation is appended to the list of outputs
    out_load.append(out)

At the end, the LUT can be stored if it did not exists and has been computed.

## Plots

We need to import plots functions from SciDataTool.

In [ ]:
from SciDataTool.Functions.Plot.plot_2D import plot_2D
from SciDataTool.Functions.Plot.plot_3D import plot_3D
#Set the resolution of the figure to make them bigger
plt.rcParams['figure.dpi'] = 150

Now we can simply plot the eficiency map using this code.

In [ ]:
plot_3D(
    Xdata=OP_matrix_MTPA[:, :, 0],  # Rotational speed
    Ydata=OP_matrix_MTPA[:, :, 3],  # Torque
    Zdata=OP_matrix_MTPA[:, :, 4], # Efficiency
    xlabel="Rotational speed",
    ylabel="Torque",
    zlabel="Efficiency",
    title="Efficiency map in torque, speed plane",
    type_plot="pcolormesh",
    is_contour=True,
    levels=[0.7,0.85,0.9,0.92,0.93,0.94,0.95],
    gamma=5
)

Also, the losses can be plotted as a function of $I_d$/$I_q$ thanks to the look up table.

In [ ]:
LUT_grid = out.simu.elec.LUT_enforced

# Get Id_min, Id_max, Iq_min, Iq_max from OP_matrix
OP_matrix = LUT_grid.get_OP_array("N0","Id","Iq")
Id_min = OP_matrix[:, 1].min()
Id_max = OP_matrix[:, 1].max()
Iq_min = OP_matrix[:, 2].min()
Iq_max = OP_matrix[:, 2].max()

nd, nq = 100, 100
Id_vect = np.linspace(Id_min, Id_max, nd)
Iq_vect = np.linspace(Iq_min, Iq_max, nq)
Id, Iq = np.meshgrid(Id_vect, Iq_vect)
Id, Iq = Id.ravel(), Iq.ravel()

# Interpolate Phid/Phiq on the refined mesh

Ploss_dqh = LUT_grid.interp_Ploss_dqh(Id, Iq, N0=1200)
dict_map = {
        "Xdata": Id.reshape((nd, nq))[0, :],
        "Ydata": Iq.reshape((nd, nq))[:, 0],
        "xlabel": "d-axis current [Arms]",
        "ylabel": "q-axis current [Arms]",
        "type_plot": "pcolormesh",
        "is_contour": True,
    }
loss_list = ["stator core",
                "rotor core",
                "joule",
                "proximity",
                "magnets"]
for i, loss in enumerate(loss_list):
    plot_3D(
            Zdata=Ploss_dqh[:, i].reshape((nd, nq)),
            zlabel=f"{loss} [W]",
            Xdata= Id.reshape((nd, nq))[0, :],
            Ydata= Iq.reshape((nd, nq))[:, 0],
            xlabel= "d-axis current [Arms]",
            ylabel= "q-axis current [Arms]",
            type_plot= "pcolormesh",
            is_contour= True,
        )

Also, the torque can be plotted as a function of $I_d$/$I_q$.

In [ ]:
# Plot torque maps
plot_3D(
    Zdata=OP_matrix_MTPA[:, :, 3],
    zlabel="Average Torque [N.m]",
    title="Torque map in dq plane",
    Xdata= I_MTPA[:, :, 0],  # Id
    Ydata= I_MTPA[:, :, 1],  # Iq
    xlabel= "d-axis current [Arms]",
    ylabel= "q-axis current [Arms]",
    type_plot= "pcolormesh",
    is_contour= True,
)

## Loss models comparison

It is possible to compare several loss models in PYLEECAN. To do this, a new simulation will be defined and run as before.


It is possible to compute the Pearson correlation coefficient between the experimental values of loss and the values calculated by PYLEECAN with respect to $B$ for each value of frequency. The results for two different loss models is given in the table below.

|frequency|Bertotti|Steinmetz|
|---|---|---|
|50|0.9966|0.9974|
|60|0.9975|0.9964|
|100|0.9973|0.9978|
|150|0.9979|0.9977|
|200|0.9981|0.9980|
|300|0.9979|0.9977|
|400|0.9987|0.9982|
|600|0.9997|0.9984|
|1000|0.9985|0.9995|
|1500|0.9967|0.9990|
|2000|0.9966|0.9989|
|mean value|0.9978|0.9981|

The Steinmetz model is slightly better in average, but the Bertotti model also gives good results.

In [ ]:
simu = Simu1(name="test_loss_models_comparison", machine=machine)


Ic = 230 * np.exp(1j * 140 * np.pi / 180)

simu.input = InputCurrent(
    Nt_tot=40 * 8,
    Na_tot=200 * 8,
    OP=OPdq(N0=1200, Id_ref=Ic.real, Iq_ref=Ic.imag),
    is_periodicity_t=True,
    is_periodicity_a=True,
)

simu.mag = MagFEMM(
    is_periodicity_a=True,
    is_periodicity_t=True,
    nb_worker=4,
    is_get_meshsolution=True,
    is_fast_draw=True,
    is_calc_torque_energy=False,
)


simu.loss = LossFEA(
    is_get_meshsolution=True,
    Tsta=100,
    model_dict={"stator core Bertotti": LossModelBertotti(group = "stator core"),
                "stator core Steinmetz": LossModelSteinmetz(group = "stator core")}
)

out = simu.run()


Then, a new output object is created as the difference between the two models defined previously. This object can then be used as any OutLossModel object to visualize the difference.

In [ ]:
out.loss.loss_list.append(out.loss.loss_list[0]-out.loss.loss_list[1])
out.loss.loss_list[-1].name = "Difference"

group_names = [
    "stator core",
    "rotor core",
    "rotor magnets"
]
loss = out.loss.loss_list[-1]
loss.get_mesh_solution().plot_contour(
    "freqs=sum",
    label=f"{loss.name} Loss",
    group_names = group_names
)

## Comparing several operating points

The losses in several operating points can also be compared by subtracting the loss outputs of several simulations. This is done in the test_loss_dq_Prius.py
